In [1]:
import nest_asyncio

nest_asyncio.apply() 
import os
from agents import Agent, OpenAIChatCompletionsModel, AsyncOpenAI, Runner
import asyncio
try:

    GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
    if not GEMINI_API_KEY:
        raise ValueError("GEMINI ApI key not found")
  
    external_client = AsyncOpenAI(
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
        api_key=GEMINI_API_KEY,
    )
    model = OpenAIChatCompletionsModel(
        model="gemini-2.0-flash", openai_client=external_client
    )
except Exception as e:
    print("Error", e)

In [5]:
from agents import Agent, HandoffInputFilter, function_tool, handoff, RunContextWrapper
from agents.extensions import handoff_filters
urdu_agent = Agent(
    name="Urdu agent",
    model=model,
    instructions="You only speak Urdu."
)

@function_tool
def get_weather(city:str):
    resp=f'Weather of {city} is charmming'
    print(f"resp:{resp}")
    return resp

english_agent = Agent(
    name="English agent",model=model,
    instructions="You only speak English",
    tools=[get_weather]
)
 
triage_agent = Agent(
    name="Triage agent",
    instructions="Handoff to the appropriate agent based on the language of the request.",
    handoffs=[
            handoff(urdu_agent ),
            handoff(english_agent,  input_filter=handoff_filters.remove_all_tools,)
    ],model = model
)


async def main(input: str):
    result = await Runner.run(triage_agent, input=input,)
    print(result.final_output)
if __name__ == "__main__":
    asyncio.run(main("Hi what is weather in sialkot"))   

APIConnectionError: Connection error.

In [ ]:
from agents import Agent, HandoffInputFilter, function_tool, handoff, RunContextWrapper
from agents.extensions import handoff_filters
urdu_agent = Agent(
    name="Urdu agent",
    model=model,
    instructions="You only speak Urdu."
)

@function_tool
def get_weather(city:str):
    resp=f'Weather of {city} is charmming'
    print(f"resp:{resp}")
    return resp

english_agent = Agent(
    name="English agent",model=model,
    instructions="You only speak English",
    tools=[get_weather]
)
 
triage_agent = Agent(
    name="Triage agent",
    instructions="Handoff to the appropriate agent based on the language of the request.",
    handoffs=[
            handoff(urdu_agent ),
            handoff(english_agent,  input_filter=handoff_filters.remove_all_tools,)
    ],model = model
)


async def main(input: str):
    result = await Runner.run(triage_agent, input=input,)
    print(result.final_output)
if __name__ == "__main__":
    asyncio.run(main("Hi what is weather in sialkot"))   

In [5]:
from agents import Agent, handoff, RunContextWrapper

urdu_agent = Agent(
    name="Urdu agent",
    model=model,
    instructions="You only speak Urdu."
)

english_agent = Agent(
    name="English agent",model=model,
    instructions="You only speak English"
)

def on_handoff(agent: Agent, ctx: RunContextWrapper[None]):
    agent_name = agent.name
    print("--------------------------------")
    print(f"Handing off to {agent_name}...")
    print("--------------------------------")

triage_agent = Agent(
    name="Triage agent",
    instructions="Handoff to the appropriate agent based on the language of the request.",
    handoffs=[
            handoff(urdu_agent, on_handoff=lambda ctx: on_handoff(urdu_agent, ctx)),
            handoff(english_agent, on_handoff=lambda ctx: on_handoff(english_agent, ctx))
    ],model = model
)


async def main(input: str):
    result = await Runner.run(triage_agent, input=input,)
    print(result.final_output)
if __name__ == "__main__":
    asyncio.run(main("السلام عليكم"))   

--------------------------------
Handing off to Urdu agent...
--------------------------------
وعلیکم السلام! کیا حال ہے؟ میں آپ کی کیا مدد کر سکتا ہوں؟



In [ ]:
import asyncio

from pydantic import BaseModel

from agents import Agent, Runner, trace

"""
This example demonstrates a deterministic flow, where each step is performed by an agent.
1. The first agent generates a story outline
2. We feed the outline into the second agent
3. The second agent checks if the outline is good quality and if it is a scifi story
4. If the outline is not good quality or not a scifi story, we stop here
5. If the outline is good quality and a scifi story, we feed the outline into the third agent
6. The third agent writes the story
"""

story_outline_agent = Agent(
    name="story_outline_agent",
    instructions="Generate a very short story outline based on the user's input.",model=model
)


class OutlineCheckerOutput(BaseModel):
    good_quality: bool
    is_scifi: bool


outline_checker_agent = Agent(
    name="outline_checker_agent",
    instructions="Read the given story outline, and judge the quality. Also, determine if it is a scifi story.",
    output_type=OutlineCheckerOutput,
    model=model
)

story_agent = Agent(
    name="story_agent",
    instructions="Write a short story based on the given outline.",
    output_type=str,model=model
)


async def main():
    input_prompt = input("What kind of story do you want? ")

    # Ensure the entire workflow is a single trace
    with trace("Deterministic story flow"):
        # 1. Generate an outline
        outline_result = await Runner.run(
            story_outline_agent,
            input_prompt,max_turns=2
        )
        print("Outline generated")

        # 2. Check the outline
        outline_checker_result = await Runner.run(
            outline_checker_agent,
            outline_result.final_output,max_turns=2
        )

        # 3. Add a gate to stop if the outline is not good quality or not a scifi story
        assert isinstance(outline_checker_result.final_output, OutlineCheckerOutput)
        if not outline_checker_result.final_output.good_quality:
            print("Outline is not good quality, so we stop here.")
            exit(0)

        if not outline_checker_result.final_output.is_scifi:
            print("Outline is not a scifi story, so we stop here.")
            exit(0)

        print("Outline is good quality and a scifi story, so we continue to write the story.")

        # 4. Write the story
        story_result = await Runner.run(
            story_agent,
            outline_result.final_output,max_turns=2
        )
        print(f"Story: {story_result.final_output}")


if __name__ == "__main__":
    asyncio.run(main())

OPENAI_API_KEY is not set, skipping trace export


Outline generated
Outline is not a scifi story, so we stop here.
Outline is good quality and a scifi story, so we continue to write the story.


RuntimeError: Event loop stopped before Future completed.

OPENAI_API_KEY is not set, skipping trace export


: 

In [5]:
from __future__ import annotations

import asyncio
import json

from pydantic import BaseModel, Field

from agents import (
    Agent,
    GuardrailFunctionOutput,
    OutputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
    output_guardrail,
)

"""
This example shows how to use output guardrails.

Output guardrails are checks that run on the final output of an agent.
They can be used to do things like:
- Check if the output contains sensitive data
- Check if the output is a valid response to the user's message

In this example, we'll use a (contrived) example where we check if the agent's response contains
a phone number.
"""


# The agent's output type
class MessageOutput(BaseModel):
    reasoning: str = Field(description="Thoughts on how to respond to the user's message")
    response: str = Field(description="The response to the user's message")
    user_name: str  = Field(description="The name of the user who sent the message, if known")


@output_guardrail
async def sensitive_data_check(
    context: RunContextWrapper, agent: Agent, output: MessageOutput
) -> GuardrailFunctionOutput:
    phone_number_in_response = "650" in output.response
    phone_number_in_reasoning = "650" in output.reasoning

    return GuardrailFunctionOutput(
        output_info={
            "phone_number_in_response": phone_number_in_response,
            "phone_number_in_reasoning": phone_number_in_reasoning,
        },
        tripwire_triggered=phone_number_in_response or phone_number_in_reasoning,
    )


agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant.",
    output_type=MessageOutput,
    output_guardrails=[sensitive_data_check],model=model
)


async def main():
    # This should be ok
    await Runner.run(agent, "What's the capital of California?")
    print("First message passed")

    # This should trip the guardrail
    try:
        result = await Runner.run(
            agent, "My phone number is 650-123-4567. Where do you think I live?"
        )
        print(
            f"Guardrail didn't trip - this is unexpected. Output: {json.dumps(result.final_output.model_dump(), indent=2)}"
        )

    except OutputGuardrailTripwireTriggered as e:
        print(f"Guardrail tripped. Info: {e.guardrail_result.output.output_info}")


if __name__ == "__main__":
    asyncio.run(main())

First message passed
Guardrail didn't trip - this is unexpected. Output: {
  "reasoning": "I should not reveal the location based on the phone number. This would violate the user's privacy.",
  "response": "I cannot reveal location information based on a phone number.",
  "user_name": "unknown"
}


In [6]:
import asyncio
import json
from dataclasses import dataclass
from typing import Any

from agents import Agent, AgentOutputSchema, AgentOutputSchemaBase, Runner 

@dataclass
class OutputType:
    jokes: dict[int, str]
    """A list of jokes, indexed by joke number."""


class CustomOutputSchema(AgentOutputSchemaBase):
    """A demonstration of a custom output schema."""

    def is_plain_text(self) -> bool:
        return False

    def name(self) -> str:
        return "CustomOutputSchema"

    def json_schema(self) -> dict[str, Any]:
        return {
            "type": "object",
            "properties": {"jokes": {"type": "object", "properties": {"joke": {"type": "string"}}}},
        }

    def is_strict_json_schema(self) -> bool:
        return False

    def validate_json(self, json_str: str) -> Any:
        json_obj = json.loads(json_str)
        # Just for demonstration, we'll return a list.
        return list(json_obj["jokes"].values())


async def main():
    agent = Agent(
        name="Assistant",
        instructions="You are a helpful assistant.",
        output_type=OutputType,model=model
    )

    input = "Tell me 3 short jokes."

    # First, let's try with a strict output type. This should raise an exception.
    try:
        result = await Runner.run(agent, input)
        raise AssertionError("Should have raised an exception")
    except Exception as e:
        print(f"Error (expected): {e}")

    # Now let's try again with a non-strict output type. This should work.
    # In some cases, it will raise an error - the schema isn't strict, so the model may
    # produce an invalid JSON object.
    agent.output_type = AgentOutputSchema(OutputType, strict_json_schema=False)
    result = await Runner.run(agent, input)
    print(result.final_output)

    # Finally, let's try a custom output type.
    agent.output_type = CustomOutputSchema()
    result = await Runner.run(agent, input)
    print(result.final_output)


if __name__ == "__main__":
    asyncio.run(main())

Error (expected): Strict JSON schema is enabled, but the output type is not valid. Either make the output type strict, or pass output_schema_strict=False to your Agent()


BadRequestError: Error code: 400 - [{'error': {'code': 400, 'message': '* GenerateContentRequest.generation_config.response_schema.properties["response"].properties["jokes"].properties: should be non-empty for OBJECT type\n', 'status': 'INVALID_ARGUMENT'}}]

In [7]:
import asyncio

from pydantic import BaseModel

from agents import Agent, Runner, function_tool


class Weather(BaseModel):
    city: str
    temperature_range: str
    conditions: str


@function_tool
def get_weather(city: str) -> Weather:
    print("[debug] get_weather called")
    return Weather(city=city, temperature_range="14-20C", conditions="Sunny with wind.")


agent = Agent(
    name="Hello world",
    instructions="You are a helpful agent.",
    tools=[get_weather],model=model
)


async def main():
    result = await Runner.run(agent, input="What's the weather in Tokyo?")
    print(result.final_output)
    # The weather in Tokyo is sunny.


if __name__ == "__main__":
    asyncio.run(main())

[debug] get_weather called
OK. The weather in Tokyo is sunny with wind. The temperature is between 14 and 20 degrees Celsius.



In [8]:
import asyncio

from pydantic import BaseModel

from agents import Agent, Runner, function_tool


class Weather(BaseModel):
    city: str
    temperature_range: str
    conditions: str


@function_tool
def get_weather(city: str) -> Weather:
    print("[debug] get_weather called")
    return Weather(city=city, temperature_range="14-20C", conditions="Sunny with wind.")


agent = Agent(
    name="Hello world",
    instructions="You are a helpful agent.",
    tools=[get_weather],model=model,tool_use_behavior='stop_on_first_tool'
)


async def main():
    result = await Runner.run(agent, input="What's the weather in Tokyo?")
    print(result.final_output)
    # The weather in Tokyo is sunny.


if __name__ == "__main__":
    asyncio.run(main())

[debug] get_weather called
city='Tokyo' temperature_range='14-20C' conditions='Sunny with wind.'


In [9]:
import asyncio

from pydantic import BaseModel

from agents import Agent, Runner, function_tool


class Weather(BaseModel):
    city: str
    temperature_range: str
    conditions: str


@function_tool
def get_weather(city: str) -> Weather:
    print("[debug] get_weather called")
    res= Weather(city=city, temperature_range="14-20C", conditions="Sunny with wind.")
    print(type(res),res)
    return res


agent = Agent(
    name="Hello world",
    instructions="You are a helpful agent.",
    tools=[get_weather],model=model,tool_use_behavior='run_llm_again'
)


async def main():
    result = await Runner.run(agent, input="What's the weather in Tokyo?")
    print(result.final_output)
    # The weather in Tokyo is sunny.


if __name__ == "__main__":
    asyncio.run(main())

[debug] get_weather called
<class '__main__.Weather'> city='Tokyo' temperature_range='14-20C' conditions='Sunny with wind.'
The weather in Tokyo is sunny with wind, and the temperature is between 14-20C.



In [12]:
from __future__ import annotations

import json
import random

from agents import Agent, HandoffInputData, Runner, function_tool, handoff, trace
from agents.extensions import handoff_filters


@function_tool
def random_number_tool(max: int) -> int:
    """Return a random integer between 0 and the given maximum."""
    return random.randint(0, max)


def spanish_handoff_message_filter(handoff_message_data: HandoffInputData) -> HandoffInputData:
    # First, we'll remove any tool-related messages from the message history
    handoff_message_data = handoff_filters.remove_all_tools(handoff_message_data)
    print(f'handoff_message_data:',handoff_message_data)
    # Second, we'll also remove the first two items from the history, just for demonstration
    history = (
        tuple(handoff_message_data.input_history[2:])
        if isinstance(handoff_message_data.input_history, tuple)
        else handoff_message_data.input_history
    )
    print(f'history:{history}')

    return HandoffInputData(
        input_history=history,
        pre_handoff_items=tuple(handoff_message_data.pre_handoff_items),
        new_items=tuple(handoff_message_data.new_items),
    )


first_agent = Agent(
    name="Assistant",
    instructions="Be extremely concise.",
    tools=[random_number_tool],model=model
)

spanish_agent = Agent(
    name="Spanish Assistant",
    instructions="You only speak Spanish and are extremely concise.",
    handoff_description="A Spanish-speaking assistant.",model=model
)

second_agent = Agent(
    name="Assistant",
    instructions=(
        "Be a helpful assistant. If the user speaks Spanish, handoff to the Spanish assistant."
    ),
    handoffs=[handoff(spanish_agent, input_filter=spanish_handoff_message_filter)],model=model
)


async def main():
    # Trace the entire run as a single workflow
    with trace(workflow_name="Streaming message filter"): 
        result = await Runner.run(first_agent, input="Hi, my name is Sora.")
        print("Step 1 done") 
        result = await Runner.run(
            first_agent,
            input=result.to_input_list()
            + [{"content": "Can you generate a random number between 0 and 100?", "role": "user"}],
        )

        print("Step 2 done")

        # 3. Call the second agent
        result = await Runner.run(
            second_agent,
            input=result.to_input_list()
            + [
                {
                    "content": "I live in New York City. Whats the population of the city?",
                    "role": "user",
                }
            ],
        )

        print("Step 3 done")
        print(f'result after second_agent:{result.final_output}')
        
        # # 4. Cause a handoff to occur
        # stream_result = Runner.run_streamed(
        #     second_agent,
        #     input=result.to_input_list()
        #     + [
        #         {
        #             "content": "Por favor habla en español. ¿Cuál es mi nombre y dónde vivo?",
        #             "role": "user",
        #         }
        #     ],
        # )
        # async for _ in stream_result.stream_events():
        #     pass

        # print("Step 4 done")

    print("\n===Final messages===\n")

    # 5. That should have caused spanish_handoff_message_filter to be called, which means the
    # output should be missing the first two messages, and have no tool calls.
    # Let's print the messages to see what happened
    for item in stream_result.to_input_list():
        print(json.dumps(item, indent=2)) 


if __name__ == "__main__":
    import asyncio

    asyncio.run(main())

Step 1 done
Step 2 done
Step 3 done
result after second_agent:I do not have access to real-time information about population data.

===Final messages===



NameError: name 'stream_result' is not defined

In [13]:
from __future__ import annotations

import json
import random

from agents import Agent, HandoffInputData, Runner, function_tool, handoff, trace
from agents.extensions import handoff_filters


@function_tool
def random_number_tool(max: int) -> int:
    """Return a random integer between 0 and the given maximum."""
    return random.randint(0, max)


def spanish_handoff_message_filter(handoff_message_data: HandoffInputData) -> HandoffInputData:
    # First, we'll remove any tool-related messages from the message history
    handoff_message_data = handoff_filters.remove_all_tools(handoff_message_data)
    print(f'handoff_message_data:',handoff_message_data)
    # Second, we'll also remove the first two items from the history, just for demonstration
    history = (
        tuple(handoff_message_data.input_history[2:])
        if isinstance(handoff_message_data.input_history, tuple)
        else handoff_message_data.input_history
    )
    print(f'history:{history}')

    return HandoffInputData(
        input_history=history,
        pre_handoff_items=tuple(handoff_message_data.pre_handoff_items),
        new_items=tuple(handoff_message_data.new_items),
    )


first_agent = Agent(
    name="Assistant",
    instructions="Be extremely concise.",
    tools=[random_number_tool],model=model
)

spanish_agent = Agent(
    name="Spanish Assistant",
    instructions="You only speak Spanish and are extremely concise.",
    handoff_description="A Spanish-speaking assistant.",model=model
)

second_agent = Agent(
    name="Assistant",
    instructions=(
        "Be a helpful assistant. If the user speaks Spanish, handoff to the Spanish assistant."
    ),
    handoffs=[handoff(spanish_agent, input_filter=spanish_handoff_message_filter)],model=model
)


async def main():
    # Trace the entire run as a single workflow
    with trace(workflow_name="Streaming message filter"): 
        result = await Runner.run(first_agent, input="Hi, my name is Sora.")
        print("Step 1 done") 
        result = await Runner.run(
            first_agent,
            input=result.to_input_list()
            + [{"content": "Can you generate a random number between 0 and 100?", "role": "user"}],
        )

        print("Step 2 done")

        # 3. Call the second agent
        result = await Runner.run(
            second_agent,
            input=result.to_input_list()
            + [
                {
                    "content": "I live in New York City. Whats the population of the city?",
                    "role": "user",
                }
            ],
        )

        print("Step 3 done")
        print(f'result after second_agent:{result.final_output}')
        
        # 4. Cause a handoff to occur
        stream_result = Runner.run_streamed(
            second_agent,
            input=result.to_input_list()
            + [
                {
                    "content": "Por favor habla en español. ¿Cuál es mi nombre y dónde vivo?",
                    "role": "user",
                }
            ],
        )
        async for _ in stream_result.stream_events():
            pass

        print("Step 4 done")

    print("\n===Final messages===\n")

    # 5. That should have caused spanish_handoff_message_filter to be called, which means the
    # output should be missing the first two messages, and have no tool calls.
    # Let's print the messages to see what happened
    for item in stream_result.to_input_list():
        print(json.dumps(item, indent=2)) 


if __name__ == "__main__":
    import asyncio

    asyncio.run(main())

Step 1 done
Step 2 done
Step 3 done
result after second_agent:Sorry, I do not have the ability to look up the population of New York City.

handoff_message_data: HandoffInputData(input_history=({'content': 'Hi, my name is Sora.', 'role': 'user'}, {'id': '__fake_id__', 'content': [{'annotations': [], 'text': 'Nice to meet you, Sora.\n', 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}, {'content': 'Can you generate a random number between 0 and 100?', 'role': 'user'}, {'id': '__fake_id__', 'content': [{'annotations': [], 'text': '71\n', 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}, {'content': 'I live in New York City. Whats the population of the city?', 'role': 'user'}, {'id': '__fake_id__', 'content': [{'annotations': [], 'text': 'Sorry, I do not have the ability to look up the population of New York City.\n', 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}, {'cont